In [52]:
import cv2 as cv
import numpy as np
from functools import cmp_to_key

In [53]:
time_region_width = 40

class Number_in_res_timebox:
    def __init__(self, number, box, is_colon=False):
        self.number = number
        self.box = box
        self.is_colon = is_colon

In [54]:
def find_region(img, template):
    w = template.shape[::-1][0]
    h = template.shape[::-1][1]
    res = cv.matchTemplate(img, template, cv.TM_CCOEFF_NORMED)
    _, _, _, max_loc = cv.minMaxLoc(res)
    return max_loc, (max_loc[0] + w, max_loc[1] + h), res


def find_boxes(img, template, threshold=0.8):
    w = template.shape[::-1][0]
    h = template.shape[::-1][1]
    res = cv.matchTemplate(img, template, cv.TM_CCOEFF_NORMED)
    detected_boxes = []
    loc = np.where(res >= threshold)
    for pt in zip(*loc[::-1]):
        detected_boxes.append((pt, (pt[0] + w, pt[1] + h)))
    return detected_boxes


def box_center(box):
    return (box[0][0] + box[1][0]) * 0.5


def compare(item1, item2):
    if box_center(item1.box) < box_center(item2.box):
        return -1
    elif box_center(item1.box) > box_center(item2.box):
        return 1
    else:
        return 0

In [55]:
image = cv.imread('results_2.jpg')
gray_img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
template = cv.imread('win_label.jpg', 0)
colon = cv.imread('colon.jpg', 0)
numbers = [cv.imread("0.jpg", 0),
           cv.imread("1.jpg", 0),
           cv.imread("2.jpg", 0),
           cv.imread("3.jpg", 0),
           cv.imread("4.jpg", 0),
           cv.imread("5.jpg", 0),
           cv.imread("6.jpg", 0),
           cv.imread("7.jpg", 0),
           cv.imread("8.jpg", 0),
           cv.imread("9.jpg", 0)]

In [56]:
top_left, bottom_right, _ = find_region(gray_img, template)
time_region = gray_img[top_left[1]: bottom_right[1], bottom_right[0]: bottom_right[0] + time_region_width]
founded_number = []

for idx, num in enumerate(numbers):
    boxes = find_boxes(time_region, num)
    for box in boxes:
        founded_number.append(Number_in_res_timebox(idx, box))
boxes = find_boxes(time_region, colon)
for box in boxes:
    founded_number.append(Number_in_res_timebox(-1, box, True))
founded_number = sorted(founded_number, key=cmp_to_key(compare))
time = ''
for num in founded_number:
    if num.is_colon:
        time += ':'
    else:
        time += str(num.number)

In [57]:
print(time)

32:46
